<a href="https://colab.research.google.com/github/hyeonji0401/CUDA_practice/blob/main/CUDA_execution_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.1 엔디비아 GPU 아키텍처
- CUDA 지원 엔비디아 GPU 아키텍처는 2008년부터 발표

- 페르미 아키텍처 (2010년)
 - CUDA 관점에서 구조가 정립된 첫번째 아키텍처
 - SM 안에 같은 크기의 정사각형 하나하나가 일종의 연산 코어이며 이는 SM으로 묶임

- 암페어 아키텍처 (2020년)
 - 스트리밍 멀티프로세서(SM)는 수많은 CUDA코어로 이루어져있으며, 코어가 모여서 그룹을 이룸
 - 캐시와 메모리 제어기도 확인 가능함


**1.스트리밍 멀티프로세서(SM:streaming processor)**
- 하나의 GPU는 여러 개의 SM을 포함
- 여러 개의 CUDA 코어를 가진 연산장치
- 아키텍처마다 SM이 포함하는 CUDA 코어 수는 다름
 - 페르미 아키텍처의 경우 하나의 SM에 32개의 CUDA 코어를 가지고 있음
- CUDA 코어 외에도 레지스터 파일, 공유 메모리, L1 캐시 등이 포함됨


**2. CUDA 코어**
- GPU의 가장 기본적인 프로세싱 유닛(processing unit)으로 연산을 수행하는 가장 기본 단위
- 내부에 FP unit, INT unit과 같이 실제 연산 유닛을 확인할 수 있음
- CUDA 프로그램의 기본 동작 단위는 스레드이므로 CUDA 코어 하나가 스레드 하나를 처리한다고 말할 수 있음
- GPU 사양에서 말하는 코어 수는 일반적으로 CUDA 코어 수를 의미함

# 6.2 CUDA 스레드 계층과 GPU 하드웨어
- CUDA 스레드 계층
 - 32개 스레드 -> 워프 -> 스레드 블록 -> 그리드

**1.그리드 -> GPU**
- CUDA 커널 수행 시 스레드 레이아웃에 따라 그리드 생성
- 그리드는 GPU를 사용하는 단위, 즉, 하나의 그리드는 한 GPU에서 실행됨
 - 여러 개의 GPU를 가진 멀티 GPU시스템이어도 하나의 그리드가 여러 개의 GPU를 동시에 사용하거나 GPU를 옮겨가며 실행될 수 없음
 - 하나의 GPU가 여러개의 그리드를 처리할 수는 있음

 => 하나의 GPU를 이용해서 여러 개의 CUDA 프로그램을 동시에 실행 가능

**2.스레드 블록 -> SM**
- 그리드에 포함된 스레드 블록은 그리드가 배정된 GPU 속 SM에 의해 처리됨

=> 스레드 블록을 처리하는 단위는 SM

=> 그리드에 속한 스레드 블록 개수는 스레드 레이아웃에 따라 달라짐

- 스레드 블록은 SM에 순차적으로 균등하게 분배되어 처리됨
- 하나의 SM에 여러 개의 블록이 할당될 수도 있음
 - SM이 갖는 자원의 양과 한 스레드 블록을 처리하기 위해 필요한 자원(레지스터, 공유 메모리 등과 같은 메모리 자원)의 양에 따라 한 SM이 동시에 처리할 수있는 스레드 블록 수 가 결정됨
- 활성 블록 : SM에 할당된 블록 중 현재 필요한 자원을 모두 할당받고 실행할 수 있는 스레드 블록
 - 활성 블록의 수는 CUDA 프로그램의 성능에 큰 영향을 주며 블록당 사용 자원 양을 조절함으로써 활성 블록 수를 증가시키는 전략을 사용하기도 함

**3.워프&스레드 -> SM 속의 CUDA 코어**
- 스레드 블록에 포함된 스레드들은 워프로 분할됨
- 워프는 32개의 스레드로 구성되어있으며 각각 CUDA 코어 하나에서 처리됨
 - 페르미 아키텍처의 경우 SM 내 CUDA 코어의 수가 32이며 16씩 두 개의 그룹으로 분리되어있음
 - 각 CUDA 코어 그룹이 하나의 워프를 처리함
 - SM 내부의 CUDA 코어 개수는 아키텍처에 따라 다르지만, 대체로 32의 배수임(워프가 32개의 스레드로 구성되었기 때문)
- 워프 속 스레드들은 하나의 명령어로 제어되며 GPU가 SIMT 아키텍처라는 말은 이에 의해 나옴
- 워프 스케줄러(warp scheduler)와 명령어 전달 유닛(dispatch unit)들이 다음에 처리할 워프를 결정하거나 명령을 내리고 각 CUDA 코어는 명령어에 따라 스레드의 작업을 처리

**스레드의 실행 문맥**
- 워프 내 스레드들은 하나의 명령어에 의해 움직이지만 각 스레드는 독립적으로 처리될 수도 있음
 - 스레드는 자신만의 실행문맥(execution context)을 가짐

 => GPU는 이러한 관점에서 SIMT라고 할 수 있음
 - SIMD : 하나의 실행 문맥 또는 하나의 스레드에서 하나의 명령어로 여러 개의 데이터를 처리
 - SIMT : 자신만의 실행 문맥을 가진 스레드가 하나의 명령어에 의해 제어됨
- 실행 문맥 : 작업 상태에 대한 기록
 - GPU의 각 스레드는 자신만의 작업 상황을 레지스터에 저장하고 있음
 - 스레드 블록 내 모든 워프가 SM 내부 레지스터 파일(레이스터들의 집합)을 나누어서 사용함

 ex) 스레드 블록 안의 스레드 개수 = 512개

     레지스터 파일 내 레지스터 개수 = 5,120개

     각 스레드는 10개의 레지스터를 사용하는 형태가 됨

=> 이러한 실행 구조는 워프 처리에 대해 무비용 문맥 교환과 워프 분기라는 두가지 특성을 만들어냄

**4.무비용 문맥 교환(Zero Context Switch Overhead)**
- 문맥 교환 : 연산 장치를 사용하는 프로세스가 교체될 때 먼저 연산 장치를 사용 중이던 프로세스의 문맥을 메모리 저장(context saving)하고, 저장되어 있던 새로 들어오는 프로세스의 문맥을 메모리에서 가져와 복원(context restoring)하는 과정을 말함
 - **CPU의 경우(교재 141pg 그림 6-7참조)**
   - p out : 코어를 사용 중인 프로세스
   - p in : 다음 차례 프로세스
   - p out 이 코어를 넘겨주어야하면 작업은 종료되어야함
   - p out 이 다시 코어를 할당받았을 때 이전 상황에 대해 알고 있어야함
   - 따라서 p out이 코어를 빼앗길 때 현재 문맥을 메모리에 저장함(문맥 저장: context saving)
   - p in 의 경우는 기존에 저장해두었던 작업 상황을 복원하고 작업을 재개함(문맥 복원: context restoring)

     => 문맥 교환(context switch) : 프로세스 교체 과정에서 발생하는 문맥 저장과 문맥 복원 과정을 묶어서 지칭하는 것

 - 문맥은 일반적으로 프로세서 또는 코어 안에 있는 레지스터 값을 말함
 - 문맥 쿄환은 많은 비용(overhead)을 요구하며 잦은 문맥 교환은 시스템의 성능을 크게 낮추기도 함

 => 일반적인 시스템에서는 잦은 문맥 교환을 되도록 방지하는 것이 좋음

 => 그러나 GPU알고리즘의 경우 코어 수보다 적게는 3~4배에서 많게는 10배 이상의 스레드를 사용하길 권장

 => GPU는 문맥 교환 비용이 0에 가까움

 - **GPU의 경우 (교재 143pg 그림 6-8참조)**
   - SM 안에는 여러 개의 워프가 있고 워프는 서로 번갈아 CUDA 코어를 사용
   - 워프가 전환 되는 과정(CUDA 코어를 사용하는 스레드 교체)에서 그에 따라 CUDA 코어가 참조하는 문맥도 바뀌어야함
  
   => 문맥 교환 발생

   - 다른 연산 장치와 달리 블록 내 스레드는 SM 레지스터 파일을 나누어 사용하기 때문에 메모리에 복사하거나 메모리에서 읽어오는 작업이 불필요함

   - 스레드가 잠시 일을 멈추더라도 스레드의 문맥(레지스터 값)은 유지되며 다시 CUDA 코어를 할당받으면 다른 작업 없이 바로 연산을 시작할 수 있음

   => 문맥 교환 부하 없음

=> 스레드가 메모리로부터 데이터를 읽어오는 과정에서 발생하는 연산 지연(latency)을 숨기기 위해 많은 수의 스레드를 사용하는 전략에서 활용됨

**5.워프 분기(warp divergence)**
- GPU에서 스레드는 자신만의 실행문맥을 가지므로 독립적으로 실행되어도 문제없는 것으로 생각되지만 워프 내 스레드들은 같은 명령어로 움직인다는 점에서 스레드는 독립적으로 움직일 수 없는 것으로 보임

**분기(if)가 있는 커널에서 워프 내 스레드들이 서로 다른 길을 가야한다면?**

분기가 있는 커널 코드의 예
```
__global__ void kernel_with_branch(int *_output)
{
  if(threadIdx.x % 2 == 0)
    _output[threadIdx.x] = 1;
  else
   _output[threadIdx.x] = 0;
}
```
- x-차원 번호가 홀수인 스레드와 짝수인 스레드가 서로 다른 명령어를 수행해야함
- 그러나 GPU는 워프에게 한번에 하나의 명령어만 지시할 수 있음

=> 각 분기를 순차적으로 처리하는 방법으로 문제 해결

- 한 쪽 분기를 처리하는 동안 다른 분기에 속하는 스레드들은 아무것도 하지 않아 이 스레드들 담당하는 CUDA 코어는 아무 작업을 하지않아 사용가능한 연산 자원이 낭비됨

=> **워프 분기** : 한 워프 내 스레드들이 서로 다른 분기를 선택하여 작업이 직렬화되어 실행되는 현상

- 분기에 2가지 경로가 있을 시 없을 때보다 대비 연산 두 배 느려짐
- 워프 내 스레드는 32개이며 최악의 경우 32개의 스레드가 모두 다른 분기를 따를 경우 최대 32배까지 연산 속도가 느려질 수 있음

=> 워프 분기는 CUDA 프로그램의 성능을 크게 떨어뜨릴 수 있으므로 피하는 것이 좋음

# 6.3 메모리 접근 대기 시간 숨기기 전략
- 일반적인 데이터 처리
 - 메모리 접근(memeory access) + 연산(computation)을 반복하는 구조
 - 메모리에 접근하는 동안 연산장치는 아무 작업을 하지 못하고 쉬게 됨

 => 메모리 접근 대기 시간(memory access latency) / 메모리 접근 지연 시간 발생
- GPU는 수만 개의 코어를 가지므로 메모리 접근 대기 시간을 최소화해야 효율을 최대화할 수 있음
 - GPU는 CPU에 비해 코어 수가 확연히 많으면 메모리 타입 또한 최신 메모리(높은 대역폭)를 사용함

 => 많은 수의 연산 코어에 데이터를 빠르게 공급하기 위한 전략

 => trade off로 CPU대비 메모리 크기는 작음
- 메모리 접근 대기 시간 방지를 위한 CUDA의 전략
 - 하드 웨어 : 고대역폭 메모리 사용
 - 소프트웨어 : CUDA 코어 수보다 많은 수의 스레드를 사용
  - 한 스레드가 메모리 대기하는 동안 다른 스레드가 CUDA 코어를 사용하게하여 CUDA 코어가 쉬지 않고 일할 수 있게 함

  => 접근 대기 시간 숨기기(latency hiding) 전략
- GPU에서 메모리 접근 대기 시간 숨기기 전략을 사용할 수 있는 이유는 '무비용 문맥 교환' 특성 때문
 - 본 전략은 스레드 사이의 잦은 전환이 필요하여 문맥 교환 비용 교환이 크다면 소용없는 전략임
- 알고리즘에 따라 연산, 메모리 접근 시간의 비중 및 상호 전환 빈도는 다를 수 있음
 - 데이터 접근 잦은(IO-bounded) 알고리즘이라면 사용하는 스레드 수를 늘리는 것이 효율적임(복잡도 대비 데이터 접근 횟수가 잦기 때문)
 - 계산 집약적(compute-bounded) 알고리즘이라면 너무 많은 스레드를 사용하면 성능에 악영향을 줄 수 있음

 => GPU가 갖는 CUDA 코어 수의 10배 내외를 기준으로 코드를 작성한 후 정상 작동 확인 후 스레드 수 등을 튜닝하는 것을 추천

# 6.4 GPU 정보 확인하기
- cudaDeviceProp : CUDA에서 GPU 속성 값을 담기 위해 사용하는 구조체 자료형
 - name : GPU이름
 - major : compute capability 주 버전
 - minor : compute capability 부 버전
 - multiprocessorCount : GPU가 가진 멀티프로세서
 - totlaGlobalMem : GPU의 global 메모리 크기

- GPU의 정보를 얻어서 cudaDeviceProp 구조체에 넣어주는 CUDA API는 cudaGetDeviceProperties()함수이며, 원형은 다음과 같음
```
cudaError_t cudaGetDeviceProperties (cudaDeviceProp* prop, int deviceID)
```
 - cudaDeviceProp* prop : 얻어온 정보를 저장할 cudaDeviceProp 구조체 변수의 주소
 - int deviceID : 정보를 얻어올 GPU번호로 0번부터 시작함
- 시스템 내부 GPU 개수는 cudaGetDeviceCount()함수를 통해 알 수 있으며 원형은 다음과 같음


```
cudaError_t cudaGetDeviceCount(int *count)
```
 - int* count 는 GPU 개수를 저장할 int형 변수의 주소

- 시스템 내 GPU 정보를 출력하는 프로그램 예시

In [ ]:
%%cuda
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include "helper_cuda.h"

#include <stdio.h>

#define _1MB (1024*1024)

void main(void){
    int ngpus;
    cudaGetDeviceCount(&ngpus); //시스템 내 GPU 개수 ngpus변수에 저장
    for(int i=0; i<ngpus; i++){
        cudaDeviceProp devProp; //cudaDeviceProp 구조체 변수 devProp 선언

        cudaGetDeviceProperties(&devProp, i); // i번째 GPU 정보를 얻어 devProp에 저장

        printf("Device %d: %s\n", i, devProp.name);  //이름
        printf("\tCompute capability: %d.%d\n", devProp.major, devProp.minor); //compute capability 주 버전
        printf("\tThe number of streaming multiprocessors: %d\n", devProp.multiProcessorCount); //SM 수
        printf("\tThe number of CUDA cores: %d\n", _ConvertSMVer2Cores(devProp.major, devProp.minor) * devProp.multiProcessorCount); //CUDA 코어 수
        printf("\tGolbal memory size: %.2f MB", (float)devProp.totalGlobalMem/ _1MB); //디바이스 메모리 크기
    }
}